In [77]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [78]:
from sklearn.preprocessing import *
from sklearn.model_selection import *

In [79]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [80]:
# df = pd.read_parquet('data.parquet')

In [81]:
df = pd.read_parquet('./parquet/kess.parquet')

Drop unnecessary columns

In [82]:
df = df.drop(['ID', 'Dt_Customer'], axis=1).reset_index(drop=True)

## Modelling

In [83]:
X = df.drop('Response', axis=1)
y = df.Response

In [84]:
skf = StratifiedKFold(n_splits=5)

In [85]:
df

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Marital_Status_Together,Marital_Status_Widow,A_Marital_Status_Married_Kidhome,A_Marital_Status_Married_Teenhome,A_Marital_Status_Single_Kidhome,A_Marital_Status_Single_Teenhome,A_Marital_Status_Together_Kidhome,A_Marital_Status_Together_Teenhome,A_Marital_Status_Widow_Kidhome,A_Marital_Status_Widow_Teenhome
0,1970,84835.0,0,0,0,189,104,379,111,189,...,False,False,0,0,0,0,0,0,0,0
1,1961,57091.0,0,0,0,464,5,64,7,0,...,False,False,0,0,0,0,0,0,0,0
2,1958,67267.0,0,1,0,134,11,59,15,2,...,False,False,0,1,0,0,0,0,0,0
3,1967,32474.0,1,1,0,10,0,1,0,0,...,True,False,0,0,0,0,1,1,0,0
4,1989,21474.0,1,0,0,6,16,24,11,0,...,False,False,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196,1976,66476.0,0,1,99,372,18,126,47,48,...,False,False,0,0,0,0,0,0,0,0
2197,1977,31056.0,1,0,99,5,10,13,3,8,...,False,False,1,0,0,0,0,0,0,0
2198,1976,46310.0,1,0,99,185,2,88,15,5,...,False,False,0,0,0,0,0,0,0,0
2199,1978,65819.0,0,0,99,267,38,701,149,165,...,False,False,0,0,0,0,0,0,0,0


In [86]:
a, b = next(skf.split(X, y))
X.loc[a]

,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,...,Marital_Status_Together,Marital_Status_Widow,A_Marital_Status_Married_Kidhome,A_Marital_Status_Married_Teenhome,A_Marital_Status_Single_Kidhome,A_Marital_Status_Single_Teenhome,A_Marital_Status_Together_Kidhome,A_Marital_Status_Together_Teenhome,A_Marital_Status_Widow_Kidhome,A_Marital_Status_Widow_Teenhome
226,1952,43776.0,1,1,9,177,2,71,3,2,...,True,False,0,0,0,0,1,1,0,0
227,1965,56046.0,0,0,9,577,0,64,0,0,...,True,False,0,0,0,0,0,0,0,0
231,1979,16653.0,1,0,10,5,7,31,15,4,...,False,False,0,0,1,0,0,0,0,0
237,1961,86429.0,0,0,10,464,28,873,29,18,...,False,False,0,0,0,0,0,0,0,0
238,1983,33569.0,1,0,10,16,2,18,3,0,...,False,False,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196,1976,66476.0,0,1,99,372,18,126,47,48,...,False,False,0,0,0,0,0,0,0,0
2197,1977,31056.0,1,0,99,5,10,13,3,8,...,False,False,1,0,0,0,0,0,0,0
2198,1976,46310.0,1,0,99,185,2,88,15,5,...,False,False,0,0,0,0,0,0,0,0
2199,1978,65819.0,0,0,99,267,38,701,149,165,...,False,False,0,0,0,0,0,0,0,0


In [87]:
y.loc[a]

226     1
227     1
231     1
237     1
238     1
       ..
2196    0
2197    0
2198    0
2199    0
2200    1
Name: Response, Length: 1760, dtype: int64

In [88]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [89]:
def skfc(X, y, classifier, folds=5):
  scores = {i: {} for i in range(folds)}
  skf = StratifiedKFold(n_splits=folds, shuffle=True)
  for i, (train, test) in enumerate(skf.split(X, y)):
    print(f'Fold #{i}')
    X_train = X.loc[train]
    y_train = y.loc[train]
    X_test = X.loc[test]
    y_test = y.loc[test]

    pf = PolynomialFeatures(interaction_only=True)
    X_train = pf.fit_transform(X_train)
    X_test = pf.transform(X_test)

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    classifier.fit(X_train, y_train)
    preds = classifier.predict(X_test)

    scores[i]['Accuracy'] = accuracy_score(y_test, preds)
    scores[i]['Precision'] = precision_score(y_test, preds)
    scores[i]['Recall'] = recall_score(y_test, preds)
    scores[i]['F1 Score'] = f1_score(y_test, preds)
    scores[i]['ROC AUC'] = roc_auc_score(y_test, preds)

    # classifiers = {
    #   'Logistic Regression': LogisticRegression(random_state=0),
    #   'SVM': SVC(random_state=0, probability=True),
    #   'Naive Bayes': GaussianNB(),
    #   'Decision Tree': DecisionTreeClassifier(random_state=0),
    #   'K-Nearest Neighbors': KNeighborsClassifier()
    # }

    # scores = {name: {} for name in classifiers}

    # for name, clf in classifiers.items():
    #   clf.fit(X_train, y_train)
    #   preds = clf.predict(X_test)

    #   scores[name]['Accuracy'] = accuracy_score(y_test, preds)
    #   scores[name]['Precision'] = precision_score(y_test, preds)
    #   scores[name]['Recall'] = recall_score(y_test, preds)
    #   scores[name]['F1 Score'] = f1_score(y_test, preds)
    #   scores[name]['ROC AUC'] = roc_auc_score(y_test, preds)
    
  scores_df = pd.DataFrame(scores).T
  return(scores_df)

In [90]:
a = skfc(X, y, KNeighborsClassifier())

Fold #0
Fold #1
Fold #2
Fold #3


Fold #4


In [94]:
a.mean()

Accuracy     0.850977
Precision    0.525940
Recall       0.142108
F1 Score     0.220928
ROC AUC      0.559289
dtype: float64

In [15]:
# classifiers = {
#     'Logistic Regression': LogisticRegression(random_state=0),
#     'SVM': SVC(random_state=0, probability=True),
#     'Naive Bayes': GaussianNB(),
#     'Decision Tree': DecisionTreeClassifier(random_state=0),
#     'K-Nearest Neighbors': KNeighborsClassifier()
# }

# scores = {name: {} for name in classifiers}
# print(classifiers.items())